In [2]:
import pandas as pd
import requests
import json
import openai
from langchain_openai import ChatOpenAI
import os
from CredentialTo.CredentialToBrokerAPI import CredentialUpstox
import settings
from datetime import datetime
import upstox_client 
from tabulate import tabulate




In [3]:
df_futureOptions = pd.DataFrame()
masterData = pd.read_json("https://assets.upstox.com/market-quote/instruments/exchange/NSE.json.gz")
# masterData = requests.get(url).json() 
df_futureOptions = pd.DataFrame.from_dict(masterData)

In [4]:
df_futureOptions.head()

,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,...,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,asset_key,underlying_key,last_trading_date,price_quote_unit
0,0.0,NCD_FO,JPYINR,NSE,1.774636e+12,CE,JPYINR,JPYINR,NCD_FO|14294,1.0,...,JPYINR 61 CE 27 MAR 26,61.00,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,NCD_FO,JPYINR,NSE,1.774636e+12,PE,JPYINR,JPYINR,NCD_FO|14295,1.0,...,JPYINR 61 PE 27 MAR 26,61.00,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NSE_EQ,SDL RJ 7.49% 2035,NSE,NaN,SG,NaN,NaN,NSE_EQ|IN2920250163,100.0,...,749RJ35,NaN,1.0,IN2920250163,NORMAL,NaN,NaN,NaN,NaN,NaN
3,NaN,NSE_EQ,SDL RJ 7.57% 2043,NSE,NaN,SG,NaN,NaN,NSE_EQ|IN2920250171,100.0,...,757RJ43,NaN,1.0,IN2920250171,NORMAL,NaN,NaN,NaN,NaN,NaN
4,0.0,NCD_FO,GBPINR,NSE,1.767033e+12,PE,GBPINR,GBPINR,NCD_FO|14277,1.0,...,GBPINR 118.25 PE 29 DEC 25,118.25,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Convert Expiry to YYYY-MM-DD format

In [5]:
# Convert expiry from epoch (ms) to datetime
df_futureOptions["expiry"] = pd.to_datetime(df_futureOptions["expiry"], unit="ms")

# Convert datetime to string format YYYY-MM-DD
df_futureOptions["expiry"] = df_futureOptions["expiry"].dt.strftime("%Y-%m-%d")


# print(df_futureOptions.dtypes)  # verify expiry is now string
pd.set_option('display.max_columns', None)

df_futureOptions.head()

,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,freeze_quantity,exchange_token,minimum_lot,tick_size,asset_type,underlying_type,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,asset_key,underlying_key,last_trading_date,price_quote_unit
0,0.0,NCD_FO,JPYINR,NSE,2026-03-27,CE,JPYINR,JPYINR,NCD_FO|14294,1.0,10000.0,14294,1.0,0.25,CUR,CUR,JPYINR 61 CE 27 MAR 26,61.00,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,NCD_FO,JPYINR,NSE,2026-03-27,PE,JPYINR,JPYINR,NCD_FO|14295,1.0,10000.0,14295,1.0,0.25,CUR,CUR,JPYINR 61 PE 27 MAR 26,61.00,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NSE_EQ,SDL RJ 7.49% 2035,NSE,NaN,SG,NaN,NaN,NSE_EQ|IN2920250163,100.0,100000.0,758718,NaN,1.00,NaN,NaN,749RJ35,NaN,1.0,IN2920250163,NORMAL,NaN,NaN,NaN,NaN,NaN
3,NaN,NSE_EQ,SDL RJ 7.57% 2043,NSE,NaN,SG,NaN,NaN,NSE_EQ|IN2920250171,100.0,100000.0,758723,NaN,1.00,NaN,NaN,757RJ43,NaN,1.0,IN2920250171,NORMAL,NaN,NaN,NaN,NaN,NaN
4,0.0,NCD_FO,GBPINR,NSE,2025-12-29,PE,GBPINR,GBPINR,NCD_FO|14277,1.0,10000.0,14277,1.0,0.25,CUR,CUR,GBPINR 118.25 PE 29 DEC 25,118.25,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Fetch NEarest Expiry date from Today

In [6]:


# Today as Timestamp
today = pd.to_datetime(datetime.today().date())

# Convert expiry_dates to datetime
expiry_dates = pd.to_datetime(df_futureOptions["expiry"], errors="coerce")

# Filter only future expiries
future_expiries = expiry_dates[expiry_dates >= today]

# Find nearest expiry
nearest_expiry = future_expiries.min()

print("Nearest expiry:", nearest_expiry)


Nearest expiry: 2025-09-19 00:00:00


In [7]:
settings.expiry_list_upstox[0]

'2025-09-16'

Filter df_futureOptions BY : ["name"] == "NIFTY" | ["segment"] == "NSE_FO" | expiry

In [8]:
df_futureOptions = df_futureOptions[ (df_futureOptions["name"] == "NIFTY") & (df_futureOptions["segment"] == "NSE_FO")  & (df_futureOptions["expiry"] == settings.expiry_list_upstox[0]) ]
df_futureOptions =  df_futureOptions.reset_index(drop=True)
df_futureOptions

,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,freeze_quantity,exchange_token,minimum_lot,tick_size,asset_type,underlying_type,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,asset_key,underlying_key,last_trading_date,price_quote_unit


Fetch NIFTY 50 instrument DATA

In [9]:
# Filter NIFTY 50 data
df_nifty50 = df_futureOptions[
    (df_futureOptions["instrument_key"] == "NSE_INDEX|Nifty 50")]

df_nifty50

,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,freeze_quantity,exchange_token,minimum_lot,tick_size,asset_type,underlying_type,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,asset_key,underlying_key,last_trading_date,price_quote_unit


Fetch NIFTY LTP 

In [10]:
instrument_key='NSE_INDEX|Nifty 50'

df_ltp = pd.DataFrame



from upstox_client.rest import ApiException
from pprint import pprint

# Configure OAuth2 access token for authorization: OAUTH2
configuration = upstox_client.Configuration()
configuration.access_token = CredentialUpstox.ACCESS_TOKEN  
# create an instance of the API class
api_instance = upstox_client.MarketQuoteV3Api(upstox_client.ApiClient(configuration))

try:
    # Market quotes and instruments - LTP quotes.
    api_response = api_instance.get_ltp(instrument_key=instrument_key)
    
    #  Convert API response to dict
    data_dict = api_response.to_dict()
    
      # Convert to DataFrame
    df_ltp = df_ltp.from_dict(data_dict["data"], orient="index").reset_index()
    df_ltp.rename(columns={"index": "instrument_key"}, inplace=True)
    print(df_ltp)
    
    # pprint(api_response)
except ApiException as e:
    print("Exception when calling MarketQuoteV3Api->get_ltp: %s\n" % e)

       instrument_key  last_price    instrument_token  ltq  volume        cp
0  NSE_INDEX:Nifty 50     25423.6  NSE_INDEX|Nifty 50    0       0  25330.25


Filter By Strike Price

In [11]:

# Extract LTP from response dict
nifty_ltp_value = float(df_ltp["last_price"])

import math

# nifty50_ltp_data = 24740.6

# Round to nearest 50
nearest_strike = round(nifty_ltp_value / 50) * 50  

# Generate 5 strikes around LTP (2 below, 2 above, + the nearest one)
strike_list = [nearest_strike + i * 50 for i in range(-3, 3)]

# Filter DataFrame
df_futureOptions = df_futureOptions[
    df_futureOptions["strike_price"].isin(strike_list)
].reset_index(drop=True)

print("LTP:", nifty_ltp_value)
print("Nearest 5 strikes:", strike_list)


LTP: 25423.6
Nearest 5 strikes: [25250, 25300, 25350, 25400, 25450, 25500]


/var/folders/bm/2sqnclys19zb2dh55g0cx4zc0000gn/T/ipykernel_6425/701431457.py:2: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  nifty_ltp_value = float(df_ltp["last_price"])


FINAL MASTERDATA

In [12]:
df_futureOptions.head(11)


,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,freeze_quantity,exchange_token,minimum_lot,tick_size,asset_type,underlying_type,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,asset_key,underlying_key,last_trading_date,price_quote_unit


In [13]:
df_futureOptions.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   weekly             0 non-null      float64
 1   segment            0 non-null      object 
 2   name               0 non-null      object 
 3   exchange           0 non-null      object 
 4   expiry             0 non-null      object 
 5   instrument_type    0 non-null      object 
 6   asset_symbol       0 non-null      object 
 7   underlying_symbol  0 non-null      object 
 8   instrument_key     0 non-null      object 
 9   lot_size           0 non-null      float64
 10  freeze_quantity    0 non-null      float64
 11  exchange_token     0 non-null      int64  
 12  minimum_lot        0 non-null      float64
 13  tick_size          0 non-null      float64
 14  asset_type         0 non-null      object 
 15  underlying_type    0 non-null      object 
 16  trading_symbol     0 non-null      obj

CONNECT TO WEBSOCKET LIVE FEED

Fetch All Instrument Keys to subscribe

In [14]:
# ActivateMarketFeed
# token_list = [99926000] is a NIFTY 50 token to get ltp of NIFTY 50 index
token_list = list(set(df_futureOptions['instrument_key'].tolist()))
print(token_list)


[]


In [15]:
df_futureOptions.columns.tolist()

['weekly',
 'segment',
 'name',
 'exchange',
 'expiry',
 'instrument_type',
 'asset_symbol',
 'underlying_symbol',
 'instrument_key',
 'lot_size',
 'freeze_quantity',
 'exchange_token',
 'minimum_lot',
 'tick_size',
 'asset_type',
 'underlying_type',
 'trading_symbol',
 'strike_price',
 'qty_multiplier',
 'isin',
 'security_type',
 'short_name',
 'asset_key',
 'underlying_key',
 'last_trading_date',
 'price_quote_unit']

Version Update : 9 sep v2 

In [16]:
import time, os, json, uuid, re, threading, queue
import pandas as pd
from typing import List, Dict, Any
from openai import OpenAI
import upstox_client
from upstox_client.rest import ApiException

# ---- Initialize ----
access_token = CredentialUpstox.ACCESS_TOKEN
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

feed_col = [
    'Token','Open','High','Low','Close','Ltp',
    'Vol','Oi','theta','delta','gamma','vega','rho','iv','vtt'
]
df_feed = pd.DataFrame(columns=feed_col)
df_feed_enriched = pd.DataFrame()

# =========================================================
# Exit Manager Config & State
# =========================================================
class ExitConfig:
    target_pct = 0.05     # 5% target
    stop_pct = 0.03       # 3% stop loss
    trailing_pct = 0.02   # 2% trailing SL
    max_holding_sec = 3600  # 1 hour max hold
    dry_run = True

open_positions: Dict[str, dict] = {}  # token → position dict
exit_queue = queue.Queue()
exit_running = False

# =========================================================
# Exit Manager Core
# =========================================================
def record_fill(order: dict):
    """Record executed order as open position."""
    token = order["token"]
    open_positions[token] = {
        "side": order["side"],
        "qty": int(order["qty"]),
        "entry_price": order.get("price") or order.get("Ltp") or 0,
        "timestamp": time.time(),
        "max_price": order.get("price") or order.get("Ltp") or 0
    }
    print(f"📌 Recorded position: {token} @ {open_positions[token]}")

def should_exit_position(pos: dict, ltp: float, cfg: ExitConfig) -> str:
    """Return exit reason or None."""
    if pos["qty"] <= 0 or ltp <= 0:
        return None
    entry = pos["entry_price"]
    pnl_pct = (ltp - entry) / entry if pos["side"] == "BUY" else (entry - ltp) / entry
    elapsed = time.time() - pos["timestamp"]

    # Target
    if pnl_pct >= cfg.target_pct:
        return "TARGET"
    # Stop loss
    if pnl_pct <= -cfg.stop_pct:
        return "STOPLOSS"
    # Trailing SL
    if pos["side"] == "BUY":
        pos["max_price"] = max(pos["max_price"], ltp)
        if ltp < pos["max_price"] * (1 - cfg.trailing_pct):
            return "TRAILING_SL"
    else:  # SELL side
        pos["max_price"] = min(pos["max_price"], ltp)
        if ltp > pos["max_price"] * (1 + cfg.trailing_pct):
            return "TRAILING_SL"
    # Time exit
    if elapsed > cfg.max_holding_sec:
        return "TIME_EXIT"
    return None

def place_exit_order_upstox(token: str, pos: dict, cfg: ExitConfig):
    """Send exit order (reverse side)."""
    side = "SELL" if pos["side"] == "BUY" else "BUY"
    qty = pos["qty"]
    if cfg.dry_run:
        print(f"💡 [DRY RUN] Exit {side} {qty} {token}")
        return {"status": "simulated", "side": side, "qty": qty, "token": token}
    try:
        configuration = upstox_client.Configuration()
        configuration.access_token = access_token
        api_client = upstox_client.ApiClient(configuration)
        order_api = upstox_client.OrderApi(api_client)

        order_req = upstox_client.PlaceOrderRequest(
            quantity=str(qty),
            product="I",  # Intraday
            validity="DAY",
            price=0.0,
            tag="EXIT-ENGINE",
            instrument_token=token,
            order_type="MARKET",
            transaction_type=side,
            disclosed_quantity=0,
            trigger_price=0,
            is_amo=False
        )
        resp = order_api.place_order(body=order_req, api_version="3.0")
        return {"status": "placed", "order_id": resp.data.order_id, "side": side, "qty": qty, "token": token}
    except ApiException as e:
        return {"status": "error", "reason": str(e)}

def enqueue_snapshot_for_exit(snapshot_df: pd.DataFrame):
    exit_queue.put(snapshot_df.copy())

def exit_worker(cfg: ExitConfig):
    global exit_running
    while exit_running:
        try:
            snapshot = exit_queue.get(timeout=1)
        except queue.Empty:
            continue
        for token, pos in list(open_positions.items()):
            if token not in snapshot["Token"].values:
                continue
            ltp = snapshot.loc[snapshot["Token"] == token, "Ltp"].iloc[0]
            reason = should_exit_position(pos, ltp, cfg)
            if reason:
                print(f"⚡ Exiting {token} due to {reason}")
                res = place_exit_order_upstox(token, pos, cfg)
                print("➡️ Exit Order:", res)
                open_positions.pop(token, None)

def start_exit_manager(cfg: ExitConfig):
    global exit_running
    exit_running = True
    t = threading.Thread(target=exit_worker, args=(cfg,), daemon=True)
    t.start()
    print("✅ Exit manager started")

def stop_exit_manager():
    global exit_running
    exit_running = False
    print("🛑 Exit manager stopped")



Tasks  - 


⚠️ Notes & Limitations

With only ₹3k, buying is the only feasible strategy (no spreads/hedges).

Requires clear downside move for large % gains.

Must monitor OI, bid-ask spreads, and underlying NIFTY closely.

Always use limit orders for better fills.


--  Add  monitor OI, bid-ask spreads, and underlying NIFTY to df_feed and into LLM Prompt also


IDEAS :

-   currently I'm calling LLM every 10s ... instead of this I can say Analyse the LIVE feed data for 15 mins ( or specific time) then create strategy and execute order . 

In [ ]:
import time
import os, json
import pandas as pd
from openai import OpenAI
import upstox_client
import re, json
import threading



# ---- Initialize ----
access_token = CredentialUpstox.ACCESS_TOKEN
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ---- Expand df_feed schema with 10 levels (adjust N as needed) ----
N_LEVELS = 30   # you can set to 5, 10, 20 depending on how much depth you want

feed_col = [
    'Token','Open','High','Low','Close','Ltp','Ltt','Ltq','Cp',
    'Vol','Oi','theta','delta','gamma','vega','rho','iv','vtt',
    'Atp','Tbq','Tsq'
]

# add bid/ask columns dynamically
for i in range(1, N_LEVELS+1):
    feed_col += [f'BidP{i}', f'BidQ{i}', f'AskP{i}', f'AskQ{i}']

df_feed = pd.DataFrame(columns=feed_col)
df_feed_enriched = pd.DataFrame()



import os, json, time, uuid
import pandas as pd
from typing import List, Dict, Any




# =========================================================
# 1. Merge df_futureOptions columns into df_feed
# =========================================================
def enrich_feed(df_feed: pd.DataFrame, df_futureOptions: pd.DataFrame, 
                cols_to_add: List[str] = None) -> pd.DataFrame:
    """
    Merge option metadata into df_feed.
    Columns added: lot_size, trading_symbol, strike_price, tick_size, instrument_type
    """
    if cols_to_add is None:
        cols_to_add = ['lot_size','trading_symbol','strike_price','tick_size','instrument_type']
    
    df_feed = df_feed.copy()
    df_futureOptions = df_futureOptions.copy()
    
    if 'Token' not in df_feed.columns:
        raise KeyError("df_feed must contain 'Token'")
    if 'instrument_key' not in df_futureOptions.columns:
        raise KeyError("df_futureOptions must contain 'instrument_key'")
    
    # Normalize keys
    df_feed['Token'] = df_feed['Token'].astype(str).str.strip()
    df_futureOptions['instrument_key'] = df_futureOptions['instrument_key'].astype(str).str.strip()
    
    # Deduplicate
    df_futureOptions = df_futureOptions.drop_duplicates(subset=['instrument_key'], keep='last')
    
    # Only keep available cols
    available = [c for c in cols_to_add if c in df_futureOptions.columns]
    if not available:
        print("[enrich_feed] Warning: no metadata columns found to merge.")
        return df_feed
    
    merged = df_feed.merge(
        df_futureOptions[['instrument_key'] + available],
        left_on='Token',
        right_on='instrument_key',
        how='left',
        validate='m:1'
    )
    
    if 'instrument_key' in merged.columns:
        merged = merged.drop(columns=['instrument_key'])
    
    # Convert dtypes
    if 'lot_size' in available:
        merged['lot_size'] = pd.to_numeric(merged['lot_size'], errors='coerce').astype('Int64')
    if 'strike_price' in available:
        merged['strike_price'] = pd.to_numeric(merged['strike_price'], errors='coerce')
    
    return merged


# =========================================================
# 2. Build LLM prompt
# =========================================================
def build_prompt_from_feed(snapshot_df: pd.DataFrame, max_rows: int = 10, depth_levels: int = 5) -> str:
    """
    Turn latest snapshot rows into a textual prompt for LLM,
    including enriched fields and configurable bid/ask depth.
    
    Args:
        snapshot_df: DataFrame containing enriched feed.
        max_rows: how many rows (options) to include in prompt.
        depth_levels: how many bid/ask levels to show per row.
    """
    rows = snapshot_df.tail(max_rows).to_dict(orient='records')
    lines = []
    for r in rows:
        # base info
        line = (
            f"Token:{r.get('Token')} sym:{r.get('trading_symbol')} type:{r.get('instrument_type')} "
            f"strike:{r.get('strike_price')} LTP:{r.get('Ltp')} IV:{r.get('iv')} OI:{r.get('Oi')} "
            f"Δ={r.get('delta')} Γ={r.get('gamma')} Θ={r.get('theta')} Vega={r.get('vega')} "
            f"lot:{r.get('lot_size')} tick:{r.get('tick_size')} "
            f"Ltt:{r.get('Ltt')} Ltq:{r.get('Ltq')} Cp:{r.get('Cp')} "
            f"Atp:{r.get('Atp')} Tbq:{r.get('Tbq')} Tsq:{r.get('Tsq')}"
        )
        # configurable bid/ask depth
        for i in range(1, depth_levels + 1):
            line += (
                f" | L{i}: Bid {r.get(f'BidP{i}')}/{r.get(f'BidQ{i}')} "
                f"Ask {r.get(f'AskP{i}')}/{r.get(f'AskQ{i}')}"
            )
        lines.append(line)

    feed_text = "\n".join(lines)
    
    prompt = f"""You are an expert Nifty 50 options strategist. Your objective is to analyze the price and volume patterns of NIFTY 50 option chain
    live data coming from websocket. identify potential buying opportunities. Utilize advanced charting tools and technical 
    indicators to scrutinize both short-term and long-term patterns, taking into account historical data and recent market movements. 
    I want intraday buying strategies with less than 3000 rupees amount and provide high profit and very low risks . Use Greeks data and market depth to deep analysis of options.

    Given the following live feed snapshot:

    {feed_text}

    Constraints:
    - Provide 1–3 legs only.
    - Total amount must not exceed 3000 rupees. 
    - Use Greeks data for advanced analysis.
    - Use Bid/Ask price levels for market depth analysis.
    - I want buying strategies only with high profit and low risk.
    - Each leg must specify token, side (BUY/SELL), qty, order_type, price, product.
    - Qty must not exceed 300 and should be a multiple of lot_size.
    - Respond ONLY in JSON with keys: strategy_id, rationale, legs.
    """
    return prompt


# =========================================================
# 3. LLM call (mock or real)
# =========================================================
def ask_llm_for_strategy(snapshot_df: pd.DataFrame, use_mock=False) -> Dict[str,Any]:
    prompt = build_prompt_from_feed(snapshot_df, max_rows=10, depth_levels=5)
    print("=== PROMPT SENT TO LLM ===\n", prompt)
    
    if use_mock:
        # Simple mock strategy for testing
        rows = snapshot_df.tail(2).to_dict(orient='records')
        legs = []
        for r in rows:
            legs.append({
                "token": r['Token'],
                "side": "SELL" if (r.get('theta', 0) < -10) else "BUY",
                "qty": int(min(300, (r.get('lot_size') or 1) * 1)),  # one lot
                "order_type": "MARKET",
                "price": None,
                "product": "NRML"
            })
        return {
            "strategy_id": str(uuid.uuid4()),
            "rationale": "Mock strategy: short theta if theta is very negative.",
            "legs": legs
        }
    else:
        # Real OpenAI call
        client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        resp = client.chat.completions.create(
            model="gpt-5-mini",
            temperature=1,
            messages=[
                {"role": "system", "content": "You are an expert trading assistant for Nifty options. You have deep knowledge of options trading strategies and human psychology."},
                {"role": "user", "content": prompt}
            ]
        )
        raw = resp.choices[0].message.content
        
        # Extract JSON substring if necessary
        import re
        match = re.search(r'(\{.*\})', raw, flags=re.S)
        if match:
            return json.loads(match.group(1))
        return json.loads(raw)


# =========================================================
# 4. Validate LLM response
# =========================================================
def validate_strategy(plan: Dict[str,Any], df_feed: pd.DataFrame) -> Dict[str,Any]:
    """
    Validate schema and enforce safety rules.
    """
    if not isinstance(plan, dict): 
        raise ValueError("Plan must be a dict")
    if 'legs' not in plan or not isinstance(plan['legs'], list):
        raise ValueError("Plan missing 'legs' list")
    
    for leg in plan['legs']:
        if not all(k in leg for k in ['token','side','qty']):
            raise ValueError(f"Leg missing keys: {leg}")
        if leg['token'] not in df_feed['Token'].values:
            raise ValueError(f"Unknown token: {leg['token']}")
        if int(leg['qty']) > 300:
            raise ValueError(f"Qty exceeds 300: {leg['qty']}")
        lot = df_feed.loc[df_feed['Token']==leg['token'],'lot_size'].iloc[0]
        if pd.notna(lot) and int(leg['qty']) % int(lot) != 0:
            raise ValueError(f"Qty {leg['qty']} not multiple of lot size {lot}")
    
    return plan


# =========================================================
# 5. Place Orders (dry run)
# =========================================================
from upstox_client.rest import ApiException

def place_orders(plan: Dict[str,Any], df_feed: pd.DataFrame, dry_run=True) -> List[Dict[str,Any]]:
    results = []

    configuration = upstox_client.Configuration()
    configuration.access_token = CredentialUpstox.ACCESS_TOKEN
    api_client = upstox_client.ApiClient(configuration)
    order_api = upstox_client.OrderApi(api_client)

    # ✅ Mapping for product
    product_map = {
        "NRML": "D",   # Delivery (normal)
        "MIS": "I",    # Intraday
        "CO": "CO",
        "OCO": "OCO",
        "MTF": "MTF"
    }

    for leg in plan['legs']:
        token = leg['token']
        qty = int(leg['qty'])
        side = leg['side'].upper()
        order_type = leg.get('order_type', 'MARKET').upper()
        product = leg.get('product', 'NRML').upper()
        price = leg.get('price') or 0.0

        # Map to Upstox enum
        product_code = product_map.get(product, "I")  # default to Delivery

        row = df_feed.loc[df_feed['Token'] == token].iloc[0]
        trading_symbol = row.get('trading_symbol')

        if qty > 300:
            results.append({"status": "rejected", "reason": "qty>300", "leg": leg})
            continue

        lot = int(row.get('lot_size')) if pd.notna(row.get('lot_size')) else None
        if lot and (qty % lot != 0):
            results.append({"status": "rejected", "reason": "qty_not_multiple_of_lot", "leg": leg})
            continue

        if dry_run:
            results.append({
                "status": "simulated",
                "token": token,
                "trading_symbol": trading_symbol,
                "qty": qty,
                "side": side,
                "order_type": order_type,
                "product": product_code
            })
            continue
        try:
            order_req = upstox_client.PlaceOrderRequest(
                quantity="75", # quantity should be string and multiple of lot size (e.g., 75 for NIFTY)
                product=product_code,
                validity="DAY",
                price=price,
                tag="LLM-STRATEGY",
                instrument_token=token,
                order_type=order_type,
                transaction_type=side,
                disclosed_quantity=0,
                trigger_price=0,
                is_amo=False
            )

            # 🔹 FIX: Add api_version
            resp = order_api.place_order(body=order_req, api_version="3.0")

            results.append({
                "status": "placed",
                "order_id": resp.data.order_id,
                "token": token,
                "trading_symbol": trading_symbol,
                "qty": qty,
                "side": side
            })
        except ApiException as e:
            results.append({"status": "error", "reason": str(e), "leg": leg})


    return results

# =========================================================
# Example Usage
# =========================================================
# Suppose df_feed and df_futureOptions are already defined in your notebook:
# df_feed = ...   # live feed DataFrame from websocket
# df_futureOptions = ...   # instrument metadata DataFrame

# 1. Enrich feed
# df_enriched = enrich_feed(df_feed, df_futureOptions)

# 2. Ask LLM
# plan = ask_llm_for_strategy(df_enriched, use_mock=True)   # use_mock=False for real call

# 3. Validate
# validated = validate_strategy(plan, df_enriched)

# 4. Place orders (dry run)
# orders = place_orders(validated, df_enriched, dry_run=True)
# print(orders)


import queue
import threading

# Shared queue for incoming snapshots
feed_queue = queue.Queue()
_last_eval = 0  # keep your eval timer

def on_message(message):
    """Non-blocking: just parse and enqueue."""
    try:
        feeds = message.get("feeds", {}) if isinstance(message, dict) else json.loads(message).get("feeds", {})
        feed_queue.put(feeds)   # enqueue raw feeds
    except Exception as e:
        print("⚠️ Failed to parse message:", e)


def feed_worker():
    global df_feed, _last_eval, df_feed_enriched

    while True:
        try:
            feeds = feed_queue.get(timeout=1)
        except queue.Empty:
            continue

        for token, feed_data in feeds.items():
            marketFF = feed_data.get("fullFeed", {}).get("marketFF", {})
            ltpc = marketFF.get("ltpc", {})
            ohlc_list = marketFF.get("marketOHLC", {}).get("ohlc", [])
            greeks = marketFF.get("optionGreeks", {})
            bidAsk = marketFF.get("marketLevel", {}).get("bidAskQuote", [])

            # intraday OHLC
            ohlc = next((item for item in ohlc_list if item.get("interval") == "1d"), {})

            row = {
                "Token": token,
                "Open": float(ohlc.get("open", 0)),
                "High": float(ohlc.get("high", 0)),
                "Low": float(ohlc.get("low", 0)),
                "Close": float(ohlc.get("close", 0)),
                "Ltp": float(ltpc.get("ltp", 0)),
                "Ltt": ltpc.get("ltt"),            # ✅ last trade time
                "Ltq": float(ltpc.get("ltq", 0)),  # ✅ last trade qty
                "Cp": float(ltpc.get("cp", 0)),    # ✅ closing price / prev close
                "Vol": float(ohlc.get("vol", 0)),
                "Oi": float(marketFF.get("oi", 0)),
                "theta": float(greeks.get("theta", 0)),
                "delta": float(greeks.get("delta", 0)),
                "gamma": float(greeks.get("gamma", 0)),
                "vega": float(greeks.get("vega", 0)),
                "rho": float(greeks.get("rho", 0)),
                "iv": float(marketFF.get("iv", 0)),
                "vtt": float(marketFF.get("vtt", 0)),
                "Atp": float(marketFF.get("atp", 0)),
                "Tbq": float(marketFF.get("tbq", 0)),
                "Tsq": float(marketFF.get("tsq", 0)),
            }

            # Add bid/ask levels
            for i in range(1, N_LEVELS + 1):
                bid = bidAsk[i - 1] if i - 1 < len(bidAsk) else {}
                row[f"BidP{i}"] = float(bid.get("bidP", 0))
                row[f"BidQ{i}"] = float(bid.get("bidQ", 0))
                row[f"AskP{i}"] = float(bid.get("askP", 0))
                row[f"AskQ{i}"] = float(bid.get("askQ", 0))

            if token in df_feed["Token"].values:
                df_feed.loc[df_feed["Token"] == token, list(row.keys())] = list(row.values())
            else:
                df_feed = pd.concat([df_feed, pd.DataFrame([row])], ignore_index=True)

        # enrich metadata
        df_feed_enriched = enrich_feed(df_feed, df_futureOptions)
         

        print("df_feed_enriched")
        # print(tabulate(df_feed_enriched.head(10), headers="keys", tablefmt="fancy_grid"))


        # every 10s → strategy pipeline
        if time.monotonic() - _last_eval > 10 and not df_feed_enriched.empty:
            try:
                snapshot_df = df_feed_enriched.tail(20)
                plan = ask_llm_for_strategy(snapshot_df, use_mock=False)
                validated = validate_strategy(plan, df_feed_enriched)
                orders = place_orders(validated, df_feed_enriched, dry_run=False)
                print("\n✅ Strategy:", json.dumps(plan, indent=2))

                if orders:
                    from IPython.display import display
                    display(pd.DataFrame(orders))

                placed_any = any(o.get("status") in ("placed", "simulated") for o in orders)
                if placed_any:
                    enqueue_snapshot_for_exit(df_feed_enriched.copy())
            except Exception as e:
                print("⚠️ Strategy generation failed:", e)

            _last_eval = time.monotonic()


# Start worker thread
threading.Thread(target=feed_worker, daemon=True).start()


def on_error(error):
    print("Error:", error)


def on_close():
    print("WebSocket closed")


# ---- Connect to Upstox ----
configuration = upstox_client.Configuration()
configuration.access_token = access_token

instrument_key = token_list  # e.g., 'NSE_INDEX|Nifty 50'
mode = 'full_d30'  # full_d30, ltpc, full, etc.

streamer = upstox_client.MarketDataStreamerV3(
    upstox_client.ApiClient(configuration),
    instrument_key,
    mode=mode
)

# Register handlers
streamer.on("message", on_message)
streamer.on("error", on_error)
streamer.on("close", on_close)

# Start exit manager (safe defaults; set .dry_run=False when you are confident)
cfg = ExitConfig()
cfg.dry_run = False
start_exit_manager(cfg)

# ✅ Run streamer in a background thread so Jupyter stays free
threading.Thread(target=streamer.connect, daemon=True).start()

print("🚀 WebSocket streamer started in background. You can continue using other cells.")


✅ Exit manager started
🚀 WebSocket streamer started in background. You can continue using other cells.


df_feed_enriched


In [18]:
pd.set_option("display.max_columns", None)   # show all columns
pd.set_option("display.width", None)         # no line wrapping
pd.set_option("display.colheader_justify", "center")  # optional: nicer headers

df_feed.head(11)

,Token,Open,High,Low,Close,Ltp,Ltt,Ltq,Cp,Vol,Oi,theta,delta,gamma,vega,rho,iv,vtt,Atp,Tbq,Tsq,BidP1,BidQ1,AskP1,AskQ1,BidP2,BidQ2,AskP2,AskQ2,BidP3,BidQ3,AskP3,AskQ3,BidP4,BidQ4,AskP4,AskQ4,BidP5,BidQ5,AskP5,AskQ5,BidP6,BidQ6,AskP6,AskQ6,BidP7,BidQ7,AskP7,AskQ7,BidP8,BidQ8,AskP8,AskQ8,BidP9,BidQ9,AskP9,AskQ9,BidP10,BidQ10,AskP10,AskQ10,BidP11,BidQ11,AskP11,AskQ11,BidP12,BidQ12,AskP12,AskQ12,BidP13,BidQ13,AskP13,AskQ13,BidP14,BidQ14,AskP14,AskQ14,BidP15,BidQ15,AskP15,AskQ15,BidP16,BidQ16,AskP16,AskQ16,BidP17,BidQ17,AskP17,AskQ17,BidP18,BidQ18,AskP18,AskQ18,BidP19,BidQ19,AskP19,AskQ19,BidP20,BidQ20,AskP20,AskQ20,BidP21,BidQ21,AskP21,AskQ21,BidP22,BidQ22,AskP22,AskQ22,BidP23,BidQ23,AskP23,AskQ23,BidP24,BidQ24,AskP24,AskQ24,BidP25,BidQ25,AskP25,AskQ25,BidP26,BidQ26,AskP26,AskQ26,BidP27,BidQ27,AskP27,AskQ27,BidP28,BidQ28,AskP28,AskQ28,BidP29,BidQ29,AskP29,AskQ29,BidP30,BidQ30,AskP30,AskQ30
